In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
import random
from scipy.spatial.distance import pdist

In [ ]:
df=pd.read_csv('/Users/heyunyu/Desktop/wd/df_model.csv')
note=pd.read_csv("/Users/heyunyu/Desktop/wd/original/note_data.csv")
req=pd.read_csv("/Users/heyunyu/Desktop/wd/original/investment_requests.csv")
df_full=pd.read_csv("/Users/heyunyu/Desktop/wd/classification.csv")
cluster=pd.read_csv("/Users/heyunyu/Desktop/wd/cluster.csv")

In [ ]:
cluster.drop_duplicates(subset=['anon_investor_account_id'],inplace=True)

In [ ]:
df=pd.merge(df,cluster[['anon_investor_account_id','cluster']],how='left',on='anon_investor_account_id')
df=df.rename(columns={'cluster':'Groups'})

# data processing

## df:  all request data with corresponding note information

In [ ]:
df.columns

## user-id  relabel  (add a new column user-id 0---3392)

In [ ]:
label=preprocessing.LabelEncoder()
map_user=dict()
df['user_id']=label.fit_transform(df['anon_investor_account_id'])
## map_user=dict(zip(label.classes_,label.transform(label.classes_)))
map_user=dict(zip(label.transform(label.classes_),label.classes_))

In [ ]:
len(map_user)

In [ ]:
label_=preprocessing.LabelEncoder()
map_user1=dict()
df_full['user_id']=label_.fit_transform(df_full['anon_investor_account_id'])
## map_user=dict(zip(label.classes_,label.transform(label.classes_)))
map_user1=dict(zip(label_.transform(label_.classes_),label_.classes_))

In [ ]:
df_full

## note data

In [ ]:
note.head()

In [ ]:
# df1=df[df['note_id']!=3074]
# df2=df[df['note_id']!=3075]

## payment type: one hot encoding

In [ ]:
pay=preprocessing.LabelEncoder()
map_pay=dict()
copy=pay.fit_transform(df['schedule_type'])
map_pay=dict(zip(pay.classes_,pay.transform(pay.classes_)))

In [ ]:
pay_=preprocessing.OneHotEncoder(sparse=False).fit_transform(copy.reshape(-1,1))

In [ ]:
map_pay

In [ ]:
payment=pd.DataFrame(pay_)
payment.columns=['is_I','is_N','is_P']
payment.head()

## Asset class: One hot encoding

In [ ]:
cl=preprocessing.LabelEncoder()
map_cl=dict()
copy_cl=cl.fit_transform(df['asset_class'])
map_cl=dict(zip(cl.classes_,cl.transform(cl.classes_)))
cl_=preprocessing.OneHotEncoder(sparse=False).fit_transform(copy_cl.reshape(-1,1))
asset=pd.DataFrame(cl_)
asset.columns=['is_com','is_con','is_legal','is_marin','is_real']
asset.head()

In [ ]:
map_cl

In [ ]:
df.shape

## also for entity type

In [ ]:
entity=preprocessing.LabelEncoder()
map_entity=dict()
copy_entity=entity.fit_transform(df['entity_type'])
map_entity=dict(zip(entity.classes_,entity.transform(entity.classes_)))
entity_=preprocessing.OneHotEncoder(sparse=False).fit_transform(copy_entity.reshape(-1,1))

entity_type=pd.DataFrame(entity_)
entity_type.columns=['is_BPDN','is_SPVM']
entity_type.head()
len(entity_type)

In [ ]:
map_entity

## payment type and asset class varibales concat with df

In [ ]:
df=pd.concat([df,payment],axis=1)
df=pd.concat([df,asset],axis=1)
df=pd.concat([df,entity_type],axis=1)

In [ ]:
df=pd.concat([df,req[['investment_request_waitlisted','investment_request_confirmed']]],axis=1)

In [ ]:
df.isnull().sum()

## find all categorical columns

In [ ]:
cat=[]
con=[]
for v in df.columns:
    if len(df[v].value_counts().index)<=15:
        cat.append(v)
#     else:
#         con.append(v)

In [ ]:
pd.unique(note['investment_increments'])

In [ ]:
con=['invest_req_amount',
 'invest_confirmed_amt',
 'offering_size',
 'minimum_investment_amount',
 'maximum_investment_amount',
 'investor_yield',
 'term']

In [ ]:
# for element in cat:
#     print(element,pd.unique(df[element]))
#     print(df[element].value_counts())

In [ ]:
# n_cols = 4
# n_rows = 4
# sns.set(style="darkgrid")
# for i in range(n_rows):
#     fg,ax = plt.subplots(nrows=1,ncols=n_cols,figsize=(18, 6))
#     for j in range(n_cols):
#         sns.countplot(x=cat[i*n_cols+j], data=df, ax=ax[j])

In [ ]:
# n_cols=4
# n_rows=2

# for i in range(n_rows):
#     fig,ax=plt.subplots(nrows=1,ncols=n_cols,figsize=(18,8))
#     for j in range(n_cols):
#         sns.violinplot(y=con[i*n_cols+j],data=df,ax=ax[j])

In [ ]:
# def plot_corr(df,size=15):
#     corr=df.corr()
#     fig,ax=plt.subplots(figsize=(size,size))
#     cax=ax.matshow(corr,cmap=plt.get_cmap("rainbow"))
#     plt.xticks(range(len(corr.columns)),corr.columns)
#     #plt.xticks(df_train.columns) plt.xticks?
#     plt.yticks(range(len(corr.columns)),corr.columns)
#     plt.colorbar(cax) ## legend
# plot_corr(df=d28)


# modeling

## select two notes as new offering
note_id=3057/3067

### first note with id 3057

In [ ]:
new1=df[df['note_id']==3057][['note_id','offering_size', 'minimum_investment_amount','maximum_investment_amount', 'investment_increments',
                   'investor_yield','term', 'is_I', 'is_N', 'is_P','is_BPDN','is_SPVM']].drop_duplicates()
df[df['note_id']==3057].asset_class.value_counts().index

In [ ]:
new1

### second note with id 3067

In [ ]:
new2=df[df['note_id']==3067][['note_id','offering_size', 'minimum_investment_amount','maximum_investment_amount', 'investment_increments',
                   'investor_yield','term', 'is_I', 'is_N', 'is_P','is_BPDN','is_SPVM']].drop_duplicates()
df[df['note_id']==3067].asset_class.value_counts().index

In [ ]:
new2

## function definition
using M distance instead of pearson correlation to get similarity

The M distance between two points is independent of the measurement unit of original data.  M distance also eliminates the effects of correlation between variables. 


M distance calculation is established on the basis of the overall sample, because of the covariance matrix,  for example, if the same two samples, in the two different population, finally calculated the M distance between two samples is often not the same, unless the two covariance matrix happened to be the same.

In the process of calculating Mdistance, it is required that the total sample number is greater than the dimension of the sample, otherwise the obtained population sample covariance matrix inverse matrix does not exist. 

due to the two reasons, we combine user history with the wholde note dataset together to calculate M distance

In [ ]:
df.columns

In [ ]:
## get individual history
def get_df(df,user_id,asset_class):
    d=df[df['user_id']==user_id]
    d=d[d['asset_class']==asset_class]
    return d

## get mahalanobis distance between offerings  
def get_dist(df,d,new_note):
    df=df[['note_id','offering_size', 'minimum_investment_amount','maximum_investment_amount', 'investment_increments',
                       'investor_yield','term', 'is_I', 'is_N', 'is_P','is_BPDN','is_SPVM']]
    d_=d[['note_id','offering_size', 'minimum_investment_amount','maximum_investment_amount', 'investment_increments',
                       'investor_yield','term', 'is_I', 'is_N', 'is_P','is_BPDN','is_SPVM']]
    note=pd.concat([new_note,d_,df],axis=0).drop_duplicates(['note_id']).set_index('note_id')
    dist=pdist(note,'mahalanobis')
    d_=d_.drop_duplicates(['note_id'])
    dist_user=pd.DataFrame(dist).iloc[:d_.shape[0],:]
    range1=dist.max()
    range0=dist.min()
    return dist_user,range1,range0
 
## get predicted request amount
def req_amount(d,dist_user):
    amount=0
    cum=0
    if dist_user.shape[0]>=3: 
        dist_user=dist_user.apply(norm_)
        d=d.drop_duplicates(['note_id'])
        for i in (range(len(dist_user))):
            index=d[['note_id']].iloc[i,:]
            sim=1-dist_user.iloc[i,:]
            invest=d[d['note_id']==int(index)]['invest_confirmed_amt'].values.sum()
            new_invest=invest*sim
            amount+=new_invest
            cum+=sim
        amount=amount/cum
        return amount
    elif d.shape[0]==0:
        return(0)
    else:
        print("Error: Data is too little!")

In [ ]:
## get group history
def get_df_group(df_full,df,user_id,asset_class):
    label=df_full['Groups'][df_full['user_id']==user_id].reset_index().iloc[0,1]
    d=df[df['Groups']==label]
    d=d[d['asset_class']==asset_class]
    return d

## normalize distance

In [ ]:
r1=14.59114254661032
r0=0.00
def norm_(x):
    return ((x-r0)/(r1-r0))

## d (user history): is generated by function, contains all history of one specified asset class (e.g. Legal) from one specified user

In [ ]:
## for user with user_id=1890 and asset_class is legal
d=get_df(df,user_id=1890,asset_class='LEGL')
d

## note: concat new note, user history and all note data together
because we want to compute the M(mahalanobis) distance for each pair of note, we get a whole data combining previous data together and drop duplicates rows

In [ ]:
df_=df[['note_id','offering_size', 'minimum_investment_amount','maximum_investment_amount', 'investment_increments',
                       'investor_yield','term', 'is_I', 'is_N', 'is_P','is_BPDN','is_SPVM']]
d_=d[['note_id','offering_size', 'minimum_investment_amount','maximum_investment_amount', 'investment_increments',
                       'investor_yield','term', 'is_I', 'is_N', 'is_P','is_BPDN','is_SPVM']]
note=pd.concat([new1,d_,df_],axis=0).drop_duplicates(['note_id']).set_index('note_id')
note

## distribution of all note normalized distance

In [ ]:
notes=df[['note_id','offering_size', 'minimum_investment_amount','maximum_investment_amount', 'investment_increments',
                       'investor_yield','term', 'is_I', 'is_N', 'is_P','is_BPDN','is_SPVM']].drop_duplicates(['note_id']).set_index('note_id')
dist=pd.DataFrame(pdist(notes,'mahalanobis'))

In [ ]:
#sns.set()
dd=1-dist.apply(norm_)
sns.distplot(dd)

## Examples
using the previously selected two notes and pick two users (user_id=1000/1890)

### for investors with investment history

In [ ]:
d=get_df(df,user_id=1000,asset_class='LEGL')
u,r1,r0=get_dist(df,d,new1)
req_amount(d,u)

In [ ]:
d=get_df(df,user_id=1890,asset_class='LEGL')
u,r1,r0=get_dist(df,d,new2)
req_amount(d,u)

In [ ]:
d=get_df(df,user_id=1000,asset_class='REAL')
u,r1,r0=get_dist(df,d,new1)
req_amount(d,u)

In [ ]:
d=get_df(df,1890,'REAL')
u,r1,r0=get_dist(df,d,new2)
req_amount(d,u)

### for investors without investment history

In [ ]:
d=get_df_group(df_full,df,200,'LEGL')
u,r1,r0=get_dist(df,d,new1)
req_amount(d,u)

In [ ]:
# end